In [0]:
## Load all the necessary packages

import numpy as np
import pandas as pd
import os
from PIL import Image
import pickle
import cv2

In [2]:
## mounting notebook on Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


### Loading and preparing the data


In [0]:

### Load all the pictures previosly proccessed to (224,224,3) and saved in a pickle format
with open("/content/drive/My Drive/Team Bergere/Optimizer/Data/optimizer/save_data/data_good1.pickle", 'rb') as f:
  unpickler = pickle.Unpickler(f)
  X_train = unpickler.load()
  
with open("/content/drive/My Drive/Team Bergere/Optimizer/Data/optimizer/save_data/data_good2.pickle", 'rb') as f:
  unpickler = pickle.Unpickler(f)
  X_train_1 = unpickler.load()
  X_train = pd.concat([X_train,X_train_1])
  del(X_train_1)
  
with open("/content/drive/My Drive/Team Bergere/Optimizer/Data/optimizer/save_data/data_good3.pickle", 'rb') as f:
  unpickler = pickle.Unpickler(f)
  X_train_1 = unpickler.load()
  X_train = pd.concat([X_train,X_train_1])
  del(X_train_1)

with open("/content/drive/My Drive/Team Bergere/Optimizer/Data/optimizer/save_data/data_good4.pickle", 'rb') as f:
  unpickler = pickle.Unpickler(f)
  X_train_1 = unpickler.load()
  X_train = pd.concat([X_train,X_train_1])
  del(X_train_1)

with open("/content/drive/My Drive/Team Bergere/Optimizer/Data/optimizer/save_data/data_extra2.pickle", 'rb') as f:
  unpickler = pickle.Unpickler(f)
  X_train_1 = unpickler.load()
  X_train = pd.concat([X_train,X_train_1])
  del(X_train_1)

with open("/content/drive/My Drive/Team Bergere/Optimizer/Data/optimizer/save_data/data_extra1.pickle", 'rb') as f:
  unpickler = pickle.Unpickler(f)
  X_train_1 = unpickler.load()
  X_train = pd.concat([X_train,X_train_1])
  del(X_train_1)

with open("/content/drive/My Drive/Team Bergere/Optimizer/Data/optimizer/save_data/data_extra3.pickle", 'rb') as f:
  unpickler = pickle.Unpickler(f)
  X_train_1 = unpickler.load()
  X_train = pd.concat([X_train,X_train_1])
  del(X_train_1)


In [0]:

## Separate pictures and labels
X_train= X_train.reset_index(drop = True)
x_train = np.array(list(X_train['pictures']))
y_train = np.array(list(X_train['final_label']))
del(X_train)


In [0]:
## Labels need to be one array for each output digit
d0 = []
d1 = []
d2 = []
d3 = []
d4 = []
d5 = []
for i in y_train:
  d0.append(i[0])
  d1.append(i[1])
  d2.append(i[2])
  d3.append(i[3])
  d4.append(i[4])
  d5.append(i[5])
  
trainY = [d0, d1, d2,d3,d4,d5]

In [0]:
x_train.shape

### Training the models

In [0]:
import os, shutil
import random
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions


In [0]:
#@title
import numpy as np
import os
# import helper
# import detection
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras import regularizers
import keras.utils as ku
from keras import backend as K
from keras import optimizers
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt

Load a imagenet pre-trained model

In [0]:
model = VGG16(include_top = False, weights = 'imagenet')





58892288/58889256 [==============================] - 1s 0us/step




In [0]:
## Freeze top layers
for layer in model.layers:
  layer.trainable = False

As it is a heavy model, we needed to train the model in several steps. Therefore, we load the last version trained to be able to continue the training.

In [0]:
from keras.models import load_model
model = load_model('/content/drive/My Drive/Team Bergere/Optimizer/Notebooks/Models/model_v3.27.h5')

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputVGGPreTrain (InputLayer)   (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
vgg16 (Model)                   multiple             14714688    inputVGGPreTrain[0][0]           
__________________________________________________________________________________________________
flatten (Flatten)               (None, 25088)        0           vgg16[1][0]                      
__________________________________________________________________________________________________
FC1_4096 (Dense)                (None, 1024)         25691136    flatten[0][0]                    
____________________________________________________________________________________________

### Model architecture

In [0]:

_, row, col, channels = x_train.shape
digLen = 6
numDigits = 11
epochs = 10
batch_size = 64
optim = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

## model architecture
preTrainModel = model
ptInput = keras.Input(shape = (row,col,channels), name  = 'inputVGGPreTrain') # keras way of creating a tensor 
pt_vgg16 = preTrainModel(ptInput)

Mout = Flatten(name = 'flatten')(pt_vgg16)
Mout = Dense(1024, activation='relu', name = 'FC1_4096')(Mout)
Mout = Dense(1024, activation='relu', name = 'FC1_512')(Mout)
Mout = Dense(512,  activation='relu', name = 'FC2_1024')(Mout)
Mout = Dropout(0.5)(Mout)

## The output of the model is
## Number of digits, Digit 1, Digit2, Digit 3, Digit 4, Digit 5]
numd_SM = Dense(digLen,    activation='softmax',name = 'num')(Mout)       # number of digits 
dig1_SM = Dense(numDigits, activation='softmax',name = 'dig1')(Mout)      # digit 1 
dig2_SM = Dense(numDigits, activation='softmax',name = 'dig2')(Mout)      # digit 2  
dig3_SM = Dense(numDigits, activation='softmax',name = 'dig3')(Mout)      # digit 3  
dig4_SM = Dense(numDigits, activation='softmax',name = 'dig4')(Mout)      # digit 4  
dig5_SM = Dense(numDigits, activation='softmax',name = 'dig5')(Mout)      # digit 5 

out = [numd_SM, dig1_SM, dig2_SM, dig3_SM, dig4_SM, dig5_SM]

## instantiating model based on input and output tensors
vggPreTrain = keras.Model(inputs = ptInput, outputs = out)

## configures the model for training
vggPreTrain.compile(loss = 'sparse_categorical_crossentropy',
                    optimizer = optim,
                    metrics=  ['accuracy'])
vggPreTrain.summary()

## creating plots, calculating metrics, saving results 
modName = 'vgg16_PreTrain'
#createSaveMetricsPlot(vggHistory, modName, data, vggPreTrain, validation=False)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputVGGPreTrain (InputLayer)   (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
vgg16 (Model)                   multiple             14714688    inputVGGPreTrain[0][0]           
__________________________________________________________________________________________________
flatten (Flatten)               (None, 25088)        0           vgg16[1][0]                      
__________________________________________________________________________________________________
FC1_4096 (Dense)                (None, 1024)         25691136    flatten[0]

KeyboardInterrupt: ignored

### Model training
Each epoch we save the model to chose later the one with better accuracy

In [0]:
from keras.models import load_model
model = load_model('/content/drive/My Drive/Team Bergere/Optimizer/Notebooks/Models/model_v3.27.h5')

digLen = 6
numDigits = 11
epochs = 10
batch_size = 64

callback = []

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = 'loss',
                                              factor = 0.1,
                                              verbose = 1,
                                              patience= 4,
                                              cooldown= 1,
                                              min_lr = 0.0001)

checkpointer = keras.callbacks.ModelCheckpoint(filepath='/content/drive/My Drive/Team Bergere/Optimizer/Notebooks/Models/model_v4.{epoch:02d}.h5',
                                               monitor='loss',
                                               save_best_only=False,
                                               verbose=2)


callback.append(checkpointer)
callback.append(reduce_lr)

## training 
vggHistory = model.fit(x = x_train,
                             y = trainY,
                             batch_size = batch_size,
                             epochs = 100,
                             verbose = 1,
                             shuffle = True,
                             #validation_data = (valdX, valdY),
                             callbacks = callback)


Epoch 1/100
   64/21000 [..............................] - ETA: 3:39 - loss: 2.2206 - num_loss: 0.1009 - dig1_loss: 0.3231 - dig2_loss: 0.4649 - dig3_loss: 0.5933 - dig4_loss: 0.3104 - dig5_loss: 0.4279 - num_acc: 0.9531 - dig1_acc: 0.9062 - dig2_acc: 0.8281 - dig3_acc: 0.8594 - dig4_acc: 0.9219 - dig5_acc: 0.8750